In [22]:
import pandas as pd
import numpy as np
import os

DATANAME = 'MiddlePhalanxTW'
eps_list = [0.005*i for i in range(1,21)]
eps_name = f" ({eps_list[0]} to {eps_list[-1]})" 
dataset_name = DATANAME
result_file = os.path.join("kfold_kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_new_' + '.csv')
df = pd.read_csv(result_file)

In [23]:
min_records = {'eTiOT': [], 'eTAOT' : []}
eps_records = {'eTiOT': [], 'eTAOT' : []}

def extract_best_eps(indices, alg):
    min_err = np.inf
    min_eps = 0
    for i in indices:
        if float(df[alg][i]) <= min_err and df['eps'][i] != 'Final error':
            min_err = float(df[alg][i])
            min_eps = float(df['eps'][i])
    return min_eps

eps_records['eTiOT'].append(extract_best_eps(range(len(df)), 'eTiOT'))
eps_records['eTAOT'].append(extract_best_eps(range(len(df)), 'eTAOT'))
eps_records['eTiOT'].append(extract_best_eps(range(1,len(df),2), 'eTiOT'))
eps_records['eTAOT'].append(extract_best_eps(range(1,len(df),2), 'eTAOT'))


In [24]:
eps_list = [0.01*i for i in range(1,11)]
eps_name = f" ({eps_list[0]} to {eps_list[-1]})"   
result_file_test = os.path.join("kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_new_' +'.csv')
col_map = {"eTiOT": "eTiOT", "eTAOT": r" eTAOT($\omega = \omega_{\text{grid}})$"}
df2 = pd.read_csv(result_file_test)

In [25]:
df2.columns

Index(['eps', 'eTiOT', 'eTAOT$(\omega = \omega_{\text{grid}} \;/\; 5)$',
       'eTAOT$(\omega = \omega_{\text{grid}})$',
       'eTAOT$(\omega = \omega_{\text{grid}} \times 5)$'],
      dtype='object')

In [26]:
for i in range(len(df2)):
    if float(df2['eps'][i]) == eps_records['eTiOT'][1]:
        eps_records['eTiOT'].append(df2['eTiOT'][i])
    if float(df2['eps'][i]) == eps_records['eTAOT'][1]:
        eps_records['eTAOT'].append(df2[df2.columns[3]][i])

In [27]:
eps_records

{'eTiOT': [0.085, 0.09, np.float64(0.4675324675324676)],
 'eTAOT': [0.05, 0.05, np.float64(0.4545454545454546)]}

In [28]:
new_rows = []

# eps for final_error (grid=0.005)
new_rows.append({
    "eps": "eps_final_error",
    "eTiOT": eps_records["eTiOT"][0],
    "eTAOT": eps_records["eTAOT"][0]
})

# final_error for grid=0.01
new_rows.append({
    "eps": "eps_final_error_grid001",
    "eTiOT": eps_records["eTiOT"][1],
    "eTAOT": eps_records["eTAOT"][1],
})

# eps for grid=0.01
new_rows.append({
    "eps": "final_error_grid001",
    "eTiOT": eps_records["eTiOT"][2],
    "eTAOT": eps_records["eTAOT"][2],
})

# append new rows after the last row
df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# save back
df.to_csv(result_file, index=False)
print("Updated result_file with final_error + eps values.")

Updated result_file with final_error + eps values.


In [2]:
print(len(X1[0]))

176


## Concatenate TiOT into overall results

In [5]:
import pandas as pd
import numpy as np
import os
eps_list = [0.01*i for i in range(1,11)]
eps_name = f" ({eps_list[0]} to {eps_list[-1]})" 
dataset_name = "ProximalPhalanxTW"
result_file = os.path.join("kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_new_' +'.csv')
TiOT_file = os.path.join("kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_freq5_TiOTonly' + '.csv')
final_file = os.path.join("kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_freq5_' + '.csv')
# Read both CSV files
allresult = pd.read_csv(result_file)
TiOTresult = pd.read_csv(TiOT_file)

# Replace TiOT column in file1 with TiOT column from file2
allresult["eTiOT"] = TiOTresult["eTiOT"]

# Save the modified file1
allresult.to_csv(final_file, index=False)

## End of concatenation

In [16]:
import numpy as np


def gaussian_mixture_timeseries(length, n_components=3, weights=None, means=None, stds=None, random_state=None):
    """
    Generate a Gaussian mixture time series.
    
    Parameters:
        length (int): Length of the time series.
        n_components (int): Number of Gaussian components.
        weights (list or None): Mixing weights (must sum to 1). If None, uniform weights are used.
        means (list or None): Means of Gaussians. If None, random values are used.
        stds (list or None): Standard deviations of Gaussians. If None, random values are used.
        random_state (int or None): Random seed for reproducibility.

    Returns:
        ts (ndarray): Generated time series of shape (length,).
        component_ids (ndarray): The component index chosen at each time step.
    """
    rng = np.random.default_rng(random_state)

    # Default weights
    if weights is None:
        weights = np.ones(n_components) / n_components
    weights = np.array(weights) / np.sum(weights)  # normalize
    
    # Default means and stds
    if means is None:
        means = rng.uniform(-5, 5, size=n_components)
    if stds is None:
        stds = rng.uniform(0.5, 2.0, size=n_components)

    # Sample component for each time step
    component_ids = rng.choice(n_components, size=length, p=weights)
    
    # Draw samples from corresponding Gaussians
    ts = rng.normal(means[component_ids], stds[component_ids])
    
    return ts, component_ids


# Example usage
ts, comps = gaussian_mixture_timeseries(2000, n_components=2, random_state=42)
ts2, comps = gaussian_mixture_timeseries(2000, n_components=2, random_state=41)




In [ ]:
import numpy as np
import ot
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import TiOT_lib
import os



(0.15022608346124952, 0.11265690000000106)

In [25]:
d0, pi, w, t = TiOT_lib.eTiOT(ts, ts2, eps = 0.1, freq = 5, solver='newton', verbose=True, timing=True)
d0, t

TiOT-BCD Algorithm converges after 31 iterations 


(0.1371113277233744, 2.4640232000000424)

In [28]:
d1, pi, w, t = TiOT_lib.eTiOT(ts, ts2, eps = 0.1, freq = 10,  verbose=2, timing=True, eta=1, init_stepsize=False, subprob_tol=10**-2)
d1, t

TiOT-BCD Algorithm converges after 41 iterations 


(0.1364609144799501, 1.1530865999999378)

In [19]:
d1, pi, t = TiOT_lib.eTAOT(ts, ts2, eps = 0.1, freq = 5,  verbose=2, timing=True)
d1, t

TAOT-BCD Algorithm converges after 22 iterations


(0.12769052169956135, 0.10527360000000385)

In [1]:
import numpy as np
import ot
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import TiOT_lib
import os

def process_data(dataset_name, start1, start2, numpoint ):
    filepath = os.path.join("time_series_kNN", dataset_name, dataset_name + "_TRAIN.txt" )

    with open(filepath, "r") as file:
        data = [line.strip().split() for line in file]

    # Convert to numerical values if needed
    data = [[float(value) for value in row] for row in data]

    X = [row[1:] for row in data]
    X1 = X[start1: start1 + numpoint]
    X2 = X[start2: start2 + numpoint]
    return X1, X2
dataset_name = 'CBF' 
X1, X2 = process_data(dataset_name, start1=0, start2=10, numpoint=10)


In [2]:
X, Y = X1[0], X1[6]

In [3]:
d0, pi, w, t = TiOT_lib.eTiOT(X, Y, eps = 0.01, freq = 20, solver='newton', verbose=2, timing=True)
d0, t

Newton Algorithm converges after 4 iterations with w = 0.5246703946039879, df = -1.0515239286940092e-11, df2 = 0.23448760380638695 and possible stepsize 1/df2 = 4.264617761311108
Newton Algorithm converges after 4 iterations with w = 0.581272706556326, df = 1.9123666174930554e-09, df2 = 0.37523836246040165 and possible stepsize 1/df2 = 2.664972721453896
Newton Algorithm converges after 4 iterations with w = 0.6265133945802858, df = 1.0934156791791994e-11, df2 = 0.4931020769462717 and possible stepsize 1/df2 = 2.027977667814528
Newton Algorithm converges after 4 iterations with w = 0.6644328171668055, df = 5.97241006650151e-13, df2 = 0.5200440523452673 and possible stepsize 1/df2 = 1.92291402139925
Newton Algorithm converges after 4 iterations with w = 0.6948452797176446, df = 7.830624169924416e-13, df2 = 0.5556611344464716 and possible stepsize 1/df2 = 1.7996579893898137
Newton Algorithm converges after 4 iterations with w = 0.7198609019326986, df = 1.476321235399647e-13, df2 = 0.59373

(np.float64(0.11657969659965604), 0.07060595601797104)

In [8]:
d1, pi, w, t = TiOT_lib.eTiOT(X, Y, eps = 0.01, eta=0.4, freq = 20, verbose=False, timing=True, submax_iter=100)
d1, t

(np.float64(0.11657997211508259), 0.3455720655620098)

In [4]:
d2, pi, w, t = TiOT_lib.eTiOT(X, Y, eps = 0.1, freq = 20, verbose=True, timing=True, submax_iter=20)
d2, t

Total subiteration needed for PGD: 19 with df = -0.0480113146621186
Total subiteration needed for PGD: 19 with df = -0.04885730901578723
Total subiteration needed for PGD: 19 with df = -0.04720987836665548
Total subiteration needed for PGD: 19 with df = -0.04519027360131247
Total subiteration needed for PGD: 19 with df = -0.043055287024125755
Total subiteration needed for PGD: 19 with df = -0.04093485593669537
TiOT-BCD Algorithm converges after 101 iterations
||g-g_old|| = 0.0062967749262412545


(0.06279226372256227, 0.05121640000000127)

In [5]:
d3, pi, w, t = TiOT_lib.eTiOT(X, Y, eps = 0.05, freq = 4, solver='PGD', verbose=True, timing=True)
d3,t

Total subiteration needed for PGD: 49 with df = -0.028703110824373747
Total subiteration needed for PGD: 49 with df = -0.028788357247891097
Total subiteration needed for PGD: 49 with df = -0.028034624328462727
Total subiteration needed for PGD: 49 with df = -0.02712501912648551
Total subiteration needed for PGD: 49 with df = -0.026154621599509317
Total subiteration needed for PGD: 49 with df = -0.02516016023047627
Total subiteration needed for PGD: 49 with df = -0.0241575637782391
Total subiteration needed for PGD: 49 with df = -0.023153598924250215
Total subiteration needed for PGD: 49 with df = -0.02215067164831322
Total subiteration needed for PGD: 49 with df = -0.02114907998204344
Total subiteration needed for PGD: 49 with df = -0.020148124279724233
Total subiteration needed for PGD: 49 with df = -0.01914667124920419
Total subiteration needed for PGD: 49 with df = -0.01814345736337799
Total subiteration needed for PGD: 49 with df = -0.017137277407998065
Total subiteration needed fo

(0.03672474723124107, 0.5282674000000043)

In [30]:
np.abs(d1 -d0), np.abs(d2-d0), np.abs(d3-d0)

(8.512607840849284e-10, 7.136090306093801e-06, 2.9745295493516233e-06)

In [34]:
def f(x):
    return x*10, x*100
f(10)[-1]

1000

In [35]:
f(20)[-1]

2000